# Convert FMI air quality data to JSON and Parquet

## ... from XML and WFS

In [1]:
from fmiopendata.wfs import download_stored_query
import pandas as pd
import numpy as np
import datetime

Let's download some [open air quality data](https://en.ilmatieteenlaitos.fi/open-data-manual-fmi-wfs-services) first.

In [2]:
query = 'urban::observations::airquality::hourly::multipointcoverage'

end_time = datetime.datetime.utcnow()
start_time = end_time - datetime.timedelta(days=7)
start_time = start_time.isoformat(timespec="seconds") + "Z"
end_time = end_time.isoformat(timespec="seconds") + "Z"

When fetching a whole month, the query will take a few minutes.

In [3]:
args = ['timeseries=True', f"starttime={start_time}", f"endtime={end_time}"]
obs = download_stored_query(query, args=args)

Then we'll pull out a complete list of all possible measurements.

In [4]:
cols = set([v for p in obs.data for v in obs.data[p]])
cols.remove('times')

And turn them into DataFrames, which we'll then merge into a single one.

In [5]:
dfs = []
for name in obs.data:
    data = {k: obs.data[name][k]['values'] for k in cols}
    mi = pd.MultiIndex.from_product([[name], obs.data[name]['times']], names=['place', 'hour'])
    df = pd.DataFrame(data=data, index=mi, columns=cols, dtype='float64')
    dfs.append(df)
df = pd.concat(dfs)
df.sample(5)
#df.dtypes

,,TRSC_PT1H_avg,CO_PT1H_avg,PM25_PT1H_avg,NO2_PT1H_avg,PM10_PT1H_avg,O3_PT1H_avg,QBCPM25_PT1H_AVG,NO_PT1H_avg,SO2_PT1H_avg,AQINDEX_PT1H_avg
place,hour,,,,,,,,,,
Helsinki Tapanila,2022-07-30 00:00:00,NaN,NaN,1.2,2.2,2.9,NaN,0.1,0.2,NaN,1.0
Kokkola Ykspihlaja,2022-08-02 14:00:00,NaN,NaN,1.5,-1.3,3.6,NaN,NaN,0.6,0.8,1.0
Pietarsaari Bottenviksvägen,2022-08-01 11:00:00,1.0,NaN,4.5,4.2,10.8,NaN,NaN,NaN,2.9,1.0
Järvenpää Helsingintie,2022-07-30 03:00:00,NaN,NaN,NaN,1.7,2.2,NaN,NaN,0.6,NaN,1.0
Espoo Leppävaara Läkkisepänkuja,2022-08-04 03:00:00,NaN,NaN,4.1,4.4,10.7,NaN,NaN,0.6,NaN,1.0


We'll attach the location metadata for the measurement points to the DataFrame.

In [6]:
df.attrs.update({'location_metadata': obs.location_metadata})

And finally, save it.

In [7]:
df.to_parquet('data/airquality.parquet')

As Parquet, one month worth of data will take about 200 kB, 
while a gzipped JSON file will be around 400 kB.

The original XML file was about 6.5 MB, gzipped 500 kB.

In [8]:
import json
import gzip

with gzip.open('data/airquality.json.gz', 'wt', encoding='utf-8') as f:
    out = {'data': obs.data, 'location_metadata': obs.location_metadata}
    json.dump(out, f, default=str)

In [14]:
import defusedxml.ElementTree as ET

with gzip.open('data/airquality.xml.gz', 'w') as f:
    f.write(ET.tostring(obs._xml))

In [15]:
!ls -Fl data/air*

-rw-rw-r-- 1 mikael mikael  66245 elo     4 14:06 data/airquality.json.gz
-rw-rw-r-- 1 mikael mikael  64827 elo     4 14:06 data/airquality.parquet
-rw-rw-r-- 1 mikael mikael 125156 elo     4 14:11 data/airquality.xml.gz
